In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab08.ok')

# Lab 8: Regularization

In this lab, you will go through the practice of using regularization to reduce overfitting. 

### Due Date ###
The assignment is due on **Monday, April 13th at 11:59pm PST**.

### Collaboration Policy
Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

In [ ]:
# Run this cell to set up your notebook
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as lm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from IPython.display import display, Latex, Markdown

In this lab, we will use the same dataset as in Lab 7. Run the following cell to load the data into a dataframe:

In [ ]:
from sklearn.datasets import load_boston
boston_data = load_boston()
#print(boston_data['DESCR']) # Uncomment this line if you are interested in what the features are

boston = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
boston.head()

Recall that in Lab 7, we started with a linear model using the full set of features from the `boston` dataframe, expanded the feature set by adding the square, square root, and hyperbolic tangent of every feature in the original dataframe, and used cross-validation to select the best first $N$ features from the expanded feature set. 

Please run the following cell to expand the feature set by adding the square, square root, and hyperbolic tangent of every feature in the original dataframe like we did in Lab 7. Now the feature matrix has 52 features.

In [ ]:
boston_with_extra_features = boston.copy()
for feature_name in boston.columns:
    boston_with_extra_features[feature_name + "^2"] = boston_with_extra_features[feature_name] ** 2
    boston_with_extra_features["sqrt" + feature_name] = np.sqrt(boston_with_extra_features[feature_name])
    boston_with_extra_features["tanh" + feature_name] = np.tanh(boston_with_extra_features[feature_name])
    
boston_with_extra_features.head(5)

In [ ]:
X = boston_with_extra_features
Y = boston_data['target']

However, recall from Lab 7 that when we plot the training and test error as we add each additional feature, our model begins to overfit. That is, even though the training error continues to decrease, the test error starts to increase. This is undesirable as the goal of machine learning is that our model should generalize well to unseen data.

In Lab 7, we used cross validation to find the best "first $N$" features. One drawback of this method is that we are not considering every subset of the features. In fact, we only compared the perfomance of models trained from 52 different feature sets. We can in theory use cross-validation on all possible subsets of the $52$ features, which will involving doing cross-validation on
$$2^{52} - 1 \approx 4.5 \times 10^{15}$$  
different feature sets, which can be computationally challenging.

Alteratively, we can use lasso regularization to perform automatic feature selection. And in general, we can use regularization to avoid overfitting. The idea of regularization is to penalize the weights $\theta$ corresponding to the features through some function $Reg(\theta)$ in the loss. Hence our objective is to compute the following:

$$
\hat{\theta} = \arg \min\limits_{\theta} \frac{1}{n}\sum\limits_{i=1}^{n} Loss(Y_i, f_{\theta}(X_i)) + \alpha Reg(\theta)
$$
where $\alpha$ is a hyperparameter. 

In the following questions we will examine two commonly used regularization functions ($d$ is the number of features used):

(1) Lasso: $\text{Reg}^{\text{lasso}}(\theta) = \sum_{i=1}^{d} |\theta_i|$

(2) Ridge: $\text{Reg}^{\text{ridge}}(\theta) = \sum_{i=1}^{d} \theta_i^2$

## Question 1

Before we fit a linear model with regularization, we need to first normalize our features. This is because if different features are on different magnitude scales, the magnitude of the corresponding weights for a good model may be very different. However, we penalize the weight for each feature equally in regularization. 

You can standardize a feature by computing: 
$$
z = \frac{x - \text{Mean}(x)}{\text{StdDev}(x)}
$$

In the cell below, complete the `normalize` function which returns the standardized version of input `data`. If $\text{StdDev}(x)$ is $0$ for a feature, fill in $0$'s for the standardized values for this feature.

<!--
BEGIN QUESTION
name: q1
-->

In [ ]:
def normalize(data):
    '''
    Args:
        data : a dataframe
    Returns:
        the normalized version of input data with NAN values filled with 0's
    '''
    ...

X_normalized = normalize(X)
X_normalized.head()

In [ ]:
ok.grade("q1");

Note that we asked you to implement the normalize function yourself above to help you gain familiarity with what is means by normalizing the data. However, for the rest of the lab, we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) function from sklearn.

Now we will split the dataset into training set and test set. Then we compute the normalized version of them.

In [ ]:
# Train & Test split
np.random.seed(41) # Do not change this line
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10)
X_train_normalized = StandardScaler().fit_transform(X_train)
X_test_normalized = StandardScaler().fit_transform(X_test)

## Ridge Regression


If we use ridge regression and assume our loss is $L_2$ loss, we solve the following optimization problem: 
$$
\hat{\theta} = \arg \min\limits_{\theta} \frac{1}{n}\sum\limits_{i=1}^{n} L_2(Y_i, f_{\theta}(X_i)) + \alpha \sum\limits_{k=1}^{d}\theta_k^2.
$$
where $\alpha$ is a nonnegative hyper-parameter. 

One nice thing about ridge linear regression with L-2 loss is that it admits a closed form solution even if $X^{\top}X$ is not full rank. The closed form solution is given by
$$
\hat{\theta} = (X^{\top}X + n\alpha I)^{-1}X^{\top}Y
$$
for the above optimization problem.

To evaluate model performance, we define RMSE in the following cell:

In [ ]:
def rmse_score(model, X, y):
    return np.sqrt(np.mean((y - model.predict(X)) ** 2))

## Question 2a

Use `sklearn.linear_model.ridge` to fit ridge linear regression models and compute the RMSE using 
**5-fold** cross-validation. 

**Note: the data should be normalized after splitting the data in general.** For this question, to normalize the data, please use the function [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Please use this function in combination with [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), and function `rmse_score` defined previously to complete the code in the cell below. 

<!--
BEGIN QUESTION
name: q2a
-->

In [ ]:
from sklearn.linear_model import Ridge

alpha = 0.1 # The regularization hyperparameter

# Initiate a model with the hyperparameter `alpha`
# Hint: the syntax should look like: Pipeline([("transformer", ..., ("LinearModel", ...)])
model = ... 

# Evaluate the RMSE of the validation error
# Hint: Don't forget to take the mean!
validation_error = ...


print("Validation RMSE:", validation_error)

In [ ]:
ok.grade("q2a");

### Hyperparameter in Ridge Linear Regression

We just fitted models using ridge regression! However, our choice of hyperparameter seems rather arbitrary. 

In the following questions, we will experiment with different choices of $\alpha$ and look at the effect of different values of $\alpha$ on the weights. 

For your convenience, we define a function `ridge_alpha` which returns the weights $\theta$ of the fitted linear model given standardized data `X` and `y` with ridge regression and the RMSE on the input data.

In [ ]:
def ridge_alpha(alpha, X, y):
    """
    Input:
        X - standardized feature matrix
        y - target vector
        alpha - regularization hyper-parameter
    Output:
        The tuple: (model_weights, rmse) where model_weights is d-dimensional array, each element 
        is rounded to the 3rd decimal. d is the dimension of each datapoint, or equivalently, the number 
        of features.
    """
    model = Ridge(alpha = alpha)
    model.fit(X, y)
    error = rmse_score(model, X, y)

    return (np.round(model.coef_, decimals = 3), error)

Let's first examine the output of ridge regression for extreme values of $\alpha$. 

To start, let's consider the case where $\alpha = 0$. In this case, the ridge linear regression objective function:
$$
\frac{1}{n}\sum\limits_{i=1}^{n} L_2(Y_i, f_{\theta}(X_i)) + \alpha \sum\limits_{i=1}^{d}\theta_k^2
$$
becomes
$$
\frac{1}{n}\sum\limits_{i=1}^{n} L_2(Y_i, f_{\theta}(X_i)),
$$
which is identical to the loss function for ordinary least squares. Hence, in this case, the linear model we fit will be identical to the one without regularization.

## Question 2b

### Question 2b(i)
Now let's consider the case where $\alpha$ is very large. In the cell below, compute the weights and RMSE of the ridge regression model when $\alpha = 10^{10}$.

<!--
BEGIN QUESTION
name: q2bi
-->

In [ ]:
...
print("weights:", ridge_alpha_10e10_weights)
print("RMSE:", ridge_alpha_10e10_rmse)

In [ ]:
ok.grade("q2bi");

### Question 2b(ii)

What do you observe from the result above? Justify your observation. Write your answers in the cell below:

<!--
BEGIN QUESTION
name: q2bii
-->

*Write your answer here, replacing this text.*

## Question 2c 

In the cell below, compute the **5-fold** cross validation error using the sklearn functionalities introduced in lecture for all the values in `alpha_arr`. Then find the $\alpha$ with the smallest cross-validation error.

<!--
BEGIN QUESTION
name: q2c
-->

In [ ]:
alpha_arr = np.linspace(0.02, 0.5, 60)
cv_errors = []

# Hint: This should be very similar to what you did in 2a.

model = ...

for alpha in alpha_arr:
    model.set_params(LinearModel__alpha=alpha)
        
    # compute the cross validation error
    cv_error = ...
    
    cv_errors.append(cv_error)
    
best_alpha_ridge = ...

print(f"The best alpha value is {best_alpha_ridge}")
print(f"Cross validation error for the best alpha value is {cv_errors[np.argmin(cv_errors)]}")

In [ ]:
ok.grade("q2c");

## Question 2d
In the cell below, plot the cross validation errors for different values of $\alpha$ for ridge regression. The x-axis should be $\alpha$ and y-axis should be the cross validation error.

<!--
BEGIN QUESTION
name: q2d
-->

In [ ]:
...
plt.show()

What do you observe from the plot above? Write your answer in the cell below.

*Write your answer here, replacing this text.*

## Lasso Regression

If we use lasso regression and assume our loss is $L_2$ loss, we solve the following optimization problem: 

$$
\hat{\theta} = \arg \min\limits_{\theta} \frac{1}{n}\sum\limits_{i=1}^{n} L_2(Y_i, f_{\theta}(X_i)) + \alpha \sum\limits_{k=1}^{d}|\theta_k|.
$$

One nice thing about lasso regression is that we can use it to select features because optimized weights for lasso regression are sparse, i.e., many of the weights are $0$'s. For more details on why this is true, please refer to Lecture 19. 

<img src="lasso.png" width=500px>

You can inspect the weights of a fitted lasso linear regression model in the cell below.

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha = 1)
model.fit(X_train_normalized, Y_train)
model.coef_

In comparison, the weights of a fitted ridge linear regression model looks like:

In [ ]:
model = Ridge(alpha = 1)
model.fit(X_train_normalized, Y_train)
model.coef_

### Hyperparameter in Lasso Linear Regression

Similar to ridge regression, as $\alpha$ goes to zero, lasso regression becomes identical to ordinary linear regression without regularization; as $\alpha$ goes to infinity, all the fitted weights become zero.  

## Question 3
Just like what we did for ridge regression, we need to tune the hyperparameter $\alpha$ to find the best $\alpha$ for lasso regression. In the cell below, use **5-fold** cross validation to find the optimal parameter $\alpha$.

**Note: When initiating your Lasso Linear Regression model, put `max_iter = 1000000` as an argument to `Lasso()`. The reason behind this is that for small regularization parameters, the Lasso Linear Regularization solver takes more iterations to converge.**

<!--
BEGIN QUESTION
name: q3
-->

In [ ]:
alpha_arr = np.linspace(0.1, 1.5, 60)
cv_errors = []
model = ...

for alpha in alpha_arr:
    model.set_params(LinearModel__alpha=alpha)

    # compute the cross validation error
    cv_error = ...
    
    cv_errors.append(cv_error)
    
best_alpha_lasso = ...

plt.plot(alpha_arr, cv_errors)
plt.title('Cross validation error over alpha')
plt.xlabel('Alpha')
plt.ylabel('Cross validation error')
plt.show()

print(f"The best alpha value is {best_alpha_lasso}")
print(f"Cross validation error for the best alpha value is {cv_errors[np.argmin(cv_errors)]}")

In [ ]:
ok.grade("q3");

## Question 4
In the previous questions, we have used cross validation to select the best $\alpha$'s for ridge regression and lasso regression. Based on this, select the best model with the smallest cross validation error out of all the models and hyperparameters we have considered (either lasso regression with its best $\alpha$ or ridge regression with its best $\alpha$). Train this model using the **entire training set** and evaluate the performance of the model on the test set.

**Note: If you are using Lasso for your final model, put `max_iter = 1000000` as an argument to `Lasso()`. The reason behind this is that for small regularization parameters, the Lasso Linear Regularization solver takes more iterations to converge.**

<!--
BEGIN QUESTION
name: q4
-->

In [ ]:
final_model = ...

# Fit `final_model`
...

# Error of prediction
test_RMSE = ...


print('test RMSE:', test_RMSE)

In [ ]:
ok.grade("q4");

## Submission

Congratulations! You are finished with this assignment. Please don't forget to submit by 11:59pm PST on Monday, 04/13!

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()